In [3]:
!pip install gdown
import gdown

url = 'https://drive.google.com/file/d/1TytoIgt7KI9Ep9bo8bs_X0HSSnBJX0oi'
output = 'sst2_dataset.zip'
gdown.download(url, output, quiet=False)

# Unzip the dataset
!unzip sst2_dataset.zip


/usr/local/lib/python3.11/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1TytoIgt7KI9Ep9bo8bs_X0HSSnBJX0oi
To: /content/sst2_dataset.zip
8.91kB [00:00, 11.7MB/s]

Archive:  sst2_dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of sst2_dataset.zip or
        sst2_dataset.zip.zip, and cannot find sst2_dataset.zip.ZIP, period.


CS22B2037
CHANTI BABU SAMBANGI


In [4]:
!file sst2_dataset.zip


sst2_dataset.zip: HTML document, ASCII text, with very long lines (2593)


In [5]:
from google.colab import files
uploaded = files.upload()


Saving sst2_sentiment_dataset.zip to sst2_sentiment_dataset.zip


In [6]:
!unzip sst2_dataset.zip

Archive:  sst2_dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of sst2_dataset.zip or
        sst2_dataset.zip.zip, and cannot find sst2_dataset.zip.ZIP, period.


In [7]:
!unzip sst2_sentiment_dataset.zip

Archive:  sst2_sentiment_dataset.zip
   creating: sst2_sentiment_dataset/
  inflating: sst2_sentiment_dataset/sst2_train.parquet  
  inflating: sst2_sentiment_dataset/sst2_valid.parquet  


In [8]:
import pandas as pd

# Read train and validation parquet files (Assuming files are named 'train.parquet' and 'validation.parquet')
train_df = pd.read_parquet('/content/sst2_sentiment_dataset/sst2_train.parquet')
val_df = pd.read_parquet('/content/sst2_sentiment_dataset/sst2_valid.parquet')

# Split 5000 samples from train for test set
test_df = train_df.sample(n=5000, random_state=42)
train_df = train_df.drop(test_df.index)

print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")


Train size: 62349, Validation size: 872, Test size: 5000


In [13]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [9]:
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Text cleaning function
def clean_text(text):
    text = text.lower()                             # Convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9\s]", '', text)       # Remove special characters
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Apply cleaning
train_df['cleaned'] = train_df['sentence'].apply(clean_text)
val_df['cleaned'] = val_df['sentence'].apply(clean_text)
test_df['cleaned'] = test_df['sentence'].apply(clean_text)

# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['cleaned'])

# Convert sentences to sequences
train_seq = tokenizer.texts_to_sequences(train_df['cleaned'])
val_seq = tokenizer.texts_to_sequences(val_df['cleaned'])
test_seq = tokenizer.texts_to_sequences(test_df['cleaned'])

# Pad sequences
max_len = 50
X_train = pad_sequences(train_seq, maxlen=max_len, padding='post', truncating='post')
X_val = pad_sequences(val_seq, maxlen=max_len)
X_test = pad_sequences(test_seq, maxlen=max_len)

y_train = train_df['label'].values
y_val = val_df['label'].values
y_test = test_df['label'].values


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=10000, output_dim=64, input_length=max_len))
rnn_model.add(SimpleRNN(64, return_sequences=False))
rnn_model.add(Dense(32, activation='relu'))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
rnn_model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_val, y_val))


Epoch 1/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5561 - loss: 0.6869 - val_accuracy: 0.5092 - val_loss: 0.7023
Epoch 2/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.5585 - loss: 0.6863 - val_accuracy: 0.5092 - val_loss: 0.7019
Epoch 3/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5576 - loss: 0.6865 - val_accuracy: 0.5092 - val_loss: 0.7021
Epoch 4/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5546 - loss: 0.6872 - val_accuracy: 0.5092 - val_loss: 0.7039
Epoch 5/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5573 - loss: 0.6866 - val_accuracy: 0.5092 - val_loss: 0.6994
Epoch 6/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5544 - loss: 0.6872 - val_accuracy: 0.5092 - val_loss: 0.7011
Epoch 7/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5590 - loss: 0.6863 - val_accuracy: 0.5092 - val_loss: 0.7009
Epoch 8/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5581 - loss: 0.6864 - val_accuracy: 0

In [21]:
from sklearn.metrics import classification_report, accuracy_score

rnn_preds = rnn_model.predict(X_test)
rnn_preds = (rnn_preds > 0.5).astype(int)

print("RNN Classification Report:")
print(classification_report(y_test, rnn_preds))
print("Accuracy:", accuracy_score(y_test, rnn_preds))


157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
RNN Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2167
           1       0.57      1.00      0.72      2833

    accuracy                           0.56      5000
   macro avg       0.28      0.50      0.36      5000
weighted avg       0.32      0.56      0.41      5000

Accuracy: 0.5648


In [22]:
from tensorflow.keras.layers import LSTM

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=10000, output_dim=64, input_length=max_len))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(32, activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
lstm_model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_val, y_val))


Epoch 1/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5575 - loss: 0.6866 - val_accuracy: 0.5092 - val_loss: 0.6970
Epoch 2/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5555 - loss: 0.6870 - val_accuracy: 0.5092 - val_loss: 0.6997
Epoch 3/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5588 - loss: 0.6863 - val_accuracy: 0.5092 - val_loss: 0.6983
Epoch 4/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5553 - loss: 0.6871 - val_accuracy: 0.5092 - val_loss: 0.6980
Epoch 5/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5580 - loss: 0.6864 - val_accuracy: 0.5092 - val_loss: 0.6980
Epoch 6/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5607 - loss: 0.6858 - val_accuracy: 0.5092 - val_loss: 0.6967
Epoch 7/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5552 - loss: 0.6871 - val_accuracy: 0.5092 - val_loss: 0.6976
Epoch 8/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5588 - loss: 0.6863 - val_accuracy: 0.

In [26]:
lstm_preds = lstm_model.predict(X_test)
lstm_preds = (lstm_preds > 0.5).astype(int)

print("LSTM Classification Report:")
print(classification_report(y_test, lstm_preds))
print("Accuracy:", accuracy_score(y_test, lstm_preds))


157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
LSTM Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2167
           1       0.57      1.00      0.72      2833

    accuracy                           0.57      5000
   macro avg       0.28      0.50      0.36      5000
weighted avg       0.32      0.57      0.41      5000

Accuracy: 0.5666


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
